In [ ]:
from ceda_sentinel.s2_ard_links import find_image_links
from ceda_sentinel.s2_ard_read_write import *

- Input a date range and a polygon layer for area(s) of interest
- Find Sentinel 2 tile names that intersect area(s) of interest
- Find CEDA Sentinel 2 ARD folders that are within that date range
- Find images that are not too cloudy overall
- Join links to intersecting images to area of interest polygons
- Clip the images that intersect the area(s) of interest polygons and save geotiff of the interecting area

In [ ]:
# The root url where CEDA S2 ARD images are found
base_url = "https://data.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_2"

In [ ]:
# The start date and end date over which to search for images
# Longer date range means more chance of cloud-free coverage, but also means longer spent searching
start_date = "2023-05-01"
end_date = "2023-05-31"

In [ ]:
# Read area of interest shapefile. In this case in the repo inputs folder
aoi_gdf = gpd.read_file("inputs/my_points.gpkg")

In [ ]:
# Define function to convert points -> buffer -> bounding box
def points_to_buffer_box(gdf, buffer_distance=500):
    """Buffer points by distance and convert to bounding boxes"""

    # Check if all geometries are Points

    if not all(gdf.geometry.geom_type == "Point"):

        print("All geometries in the GeoDataFrame must be Points. Exiting.")

        return None

    # Reproject to EPSG 27700

    gdf = gdf.to_crs(epsg=27700)

    # Buffer the geometries

    gdf["geometry"] = gdf.geometry.buffer(buffer_distance)

    # Convert buffers to bounding boxes

    gdf["geometry"] = gdf.geometry.apply(lambda geom: box(*geom.bounds))

    return gdf

In [ ]:
aoi_box_gdf = points_to_buffer_box(aoi_gdf, buffer_distance=500)

In [ ]:
# Process the AOI polygons:
# 1. Find a list of S2 image tile names that intersect the AOI
# 2. Find all CEDA S2 XML metadata file links within the date range containing those tile names.
# 3. Read each image metadata XML and if image not too cloudy extract image extent.
# 4. Spatial join image extent to AOI polygons. If more than one matching image, multiple rows are created in resulting geodataframe.
# Cloud Cover in the XML metadata seems to be in the 0 - 1 range, so default of 0.4 implies must be 40% or less to retain the image
aoi_image_gdf = find_image_links(
    aoi_box_gdf, start_date, end_date, base_url, cloud_cover_max=0.1
)

In [ ]:
# Plot one of the images for one AOI
plot_sample_image(aoi_image_gdf, plot_row=2)

In [ ]:
# Write all window images to disk, by default they will be written to the outputs folder in this repo
write_s2_windows_to_tif(
    aoi_image_gdf, band_idx_list=[1, 2, 3], aoi_id_column="image_id"
)

In [ ]:
images = Path.glob(Path.cwd() / "outputs", "*.tif")

In [ ]:
test = images.__next__()
with rio.open(test) as f:
    img_arr = f.read()

In [ ]:
show(img_arr[[2, 1, 0], :, :], adjust=True)